In [ ]:
import os
import pickle
import numpy as np
import matplotlib as mpl

from easydict import EasyDict
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as Data
from scipy.io import loadmat
import scipy.io as sio

In [ ]:
# parameter
batch_size = 8
opts = EasyDict()
opts.input_size = 2
opts.output_size = 1
opts.hidden_size = 128
opts.num_layers = 2
opts.nums_epoch = 100
opts.dropout = 0.5 
opts.isBidirectional = True;
opts.layer_size = 2
opts.sequence_length = 300
opts.attention_size = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

name_labels = ["noise"]
# ,"noise","differentplace"  "tabish" "john"
def ZscoreNormalization(x):
    """Z-score normaliaztion"""
    std = np.std(x)
    mean = np.mean(x)
    x = (x - np.mean(x)) / np.std(x)
    return x,mean,std
file_power_name = []
file_HR_name = []
for i in range(1,11):
    name = 'radar_'+ str(i).rjust(2,'0') + '.csv'
    file_power_name.append(name)
print(file_power_name)
for i in range(1,11):
    name = 'HR'+ str(i).rjust(2,'0') + '.npy'
    file_HR_name.append(name)   
print(file_HR_name)

para_normal_sep = 6
para_normal_jian = 15

In [ ]:
# LSTM model
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()
        self.lstm = torch.nn.LSTM(input_size = opts.input_size,hidden_size = opts.hidden_size, num_layers = opts.num_layers,
                                  batch_first = True,bidirectional = opts.isBidirectional,dropout = opts.dropout)
        for name, param in self.lstm.named_parameters():
#             if 'bias' in name:
#                 nn.init.constant_(param, 0.0)
            if 'weight' in name:
                nn.init.xavier_uniform_(param)
        self.out = torch.nn.Linear(opts.hidden_size*2, opts.output_size)
    
    def forward(self, sequence):
        r_out,_ = self.lstm(sequence)
        out = self.out(r_out)
        return out
# read power file
def readPowerDir(dirPath,pathArray,res):
    if dirPath[-1] == '/':
        return
    flag = False
    allFiles = []
    if os.path.isdir(dirPath):
        fileList = os.listdir(dirPath)
        fileList.sort()
        for f in fileList:
            if f in file_power_name:
                flag = True
            f = dirPath+'/'+f
            if os.path.isdir(f):
                subFiles = readPowerDir(f,pathArray,res)
                allFiles = subFiles + allFiles 
            else:
                if flag:
                    
                    pathArray.append(f)
                    csv_data = pd.read_csv(f,engine='python',header=None)
                    data = np.array(csv_data)
                    for i in range(data.shape[0]):
                        data[:,1],_,_ = ZscoreNormalization(data[:,1])
                        data[:,2],_,_ = ZscoreNormalization(data[:,2])
                    res.append(data)
                    allFiles.append(f)
                    flag = False
    return allFiles

# read HR file
def readHRDir(dirPath,pathArray,res):
    if dirPath[-1] == '/':
        return
    flag = False
    allFiles = []
    if os.path.isdir(dirPath):
        fileList = os.listdir(dirPath)
        fileList.sort()
        for f in fileList:
            if f in file_HR_name:
                flag = True
            f = dirPath+'/'+f
            if os.path.isdir(f):
                subFiles = readHRDir(f,pathArray,res)
                allFiles = subFiles + allFiles 
            else:
                if flag:
                    
                    pathArray.append(f)
                    data = np.load(f)
#                     for i in range(data.shape[0]):
#                         data[:,1],_,_ = ZscoreNormalization(data[:,1])
#                         data[:,0],_,_ = ZscoreNormalization(data[:,0])
                    res.append(data)
                    allFiles.append(f)
                    flag = False
    return allFiles



In [ ]:
average = []
average_fix = []
average_amb = []
# Cross-validation len(name_labels)
for i in range(len(name_labels)-1):
    name_mask = []
    name_mask.append(name_labels[i])
    # read power and distance from file according ro name_labels as trainset
    PowerTrain = np.array([0])
    HRTrain = np.array([0])
    for name in name_labels:
        if(name in name_mask):
            continue
        pathArray =  []
        power = []
        HR = []
        path = "/.../.../RobustVSDataset/"+name
        res = readPowerDir(path,pathArray,power)
        pathArray =  []
        res = readHRDir(path,pathArray,HR)
        power = np.array(power)
        HR = np.array(HR)
        PowerTrain = power if  PowerTrain.any() == 0 else np.concatenate((PowerTrain, power), axis = 0)
        HRTrain = HR if  HRTrain.any() == 0 else np.concatenate((HRTrain, HR), axis = 0)
    # read power and distance from file according ro name_mask as testset
    PowerTest = np.array([0])
    HRTest = np.array([0])    
    for name in name_mask:
        pathArray =  []
        power = []
        HR = []
        path = "/.../.../RobustVSDataset/"+ name
        res = readPowerDir(path,pathArray,power)
        pathArray =  []
        res = readHRDir(path,pathArray,HR)
        power = np.array(power)
        HR = np.array(HR)
        PowerTest = power if  PowerTest.any() == 0 else np.concatenate((PowerTest, power), axis = 0)
        HRTest = HR if  HRTest.any() == 0 else np.concatenate((HRTest, HR), axis = 0)
#     print("Read file success!")
    
    
    model = LSTM()
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)   # optimize parameters
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.8,weight_decay=1e-4)
    loss_func = nn.MSELoss()

    inputs, targets = torch.from_numpy(PowerTrain[:,:,1:]).type(torch.FloatTensor),torch.from_numpy(HRTrain[:,:,0]).type(torch.FloatTensor)
    inputs, targets = inputs.to(device), targets.to(device)
    torch_dataset = Data.TensorDataset(inputs,targets)
    loader = Data.DataLoader(
                    dataset=torch_dataset,      # torch TensorDataset format
                    batch_size=batch_size,      # mini batch size
                    shuffle=True,                            
                )
    train_loss, train_count, val_loss, val_count = 0,0,0,0
    val_inputs,val_targets  = torch.from_numpy(PowerTest[:,:,1:]).type(torch.FloatTensor),torch.from_numpy(HRTest[:,:,0]).type(torch.FloatTensor)
    val_inputs,val_targets = val_inputs.to(device), val_targets.to(device)
    val_inputs = val_inputs.permute(1, 0, 2)
    val_targets = val_targets.t()
    val_targets = val_targets[:,:,np.newaxis]
#   begin train
    for epoch in range(0, opts.nums_epoch+1):
        for step, (x, y) in enumerate(loader):
            model.zero_grad()
            model.train()
            x = x.permute(1, 0, 2)
            y = y.t()
            y = y[:,:,np.newaxis]
            prediction= model(x)
            loss = loss_func(prediction, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step() 
            train_loss += loss.item()
            train_count += 1

        model.eval()
        with torch.no_grad():
            val_predict = model(val_inputs)
            loss_ = loss_func(val_predict, val_targets)
            val_loss += loss_.item()
            val_count += 1
        if(epoch% 50 == 0):
            print('Epoch %d / %d     Train loss: %f     val loss: %f'% (epoch,opts.nums_epoch,  train_loss/train_count,val_loss/val_count)) 
    val_predict = model(val_inputs)
    sum = 0
    sum_fix = 0
    sum_amb = 0
    for i in range(val_predict.shape[1]):
        if name_mask[0] == "jian":
            para = para_normal_jian
        else:
            para = para_normal_sep
        if(i < para):
            sum_amb += torch.sum(torch.abs(val_predict[:,i]-val_targets[:,i]))/300
        else:
            sum_fix += torch.sum(torch.abs(val_predict[:,i]-val_targets[:,i]))/300
        sum += torch.sum(torch.abs(val_predict[:,i]-val_targets[:,i]))/300
        print('index:%d    MSE: %f '%(i,torch.sum(torch.abs(val_predict[:,i]-val_targets[:,i]))/300))
        average.append(torch.sum(torch.abs(val_predict[:,i]-val_targets[:,i])).item()/300)
    print('Test: %s  average MSE: %f' % (name_mask[0],sum/val_predict.shape[1] ))
                                                                            
   
    
# print("average: %f" %(np.mean(average)))
# print("average fix: %f" %(np.mean(average_fix)))
# print("average amb: %f" %(np.mean(average_amb)))
print(average)

np.savetxt("generation_HR_noise.csv", average)
print(average_fix)
print(average_amb)